In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NearestNeighbour
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation 
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np

### testing outputs with manually-built rss-loc database

this *_data_dictionary was built manually

we measured RSS values at chosen locations and saved them (the values are not pulled from the experiment data.json)

In [2]:
exp001_nearestneighbour_db_manual_data_dictionary = {
    '0_0': [-34, -63, -52],
    '0_7': [-50, -71, -70],
    '-6_7': [-45, -66, -66],
    '6_7': [-42, -65, -66],
    '-14_2': [-41, -71, -70],
    '14_2': [-48, -53, -53],
    '-28_-4': [-62, -71, -44],
    '-28_4': [-53, -73, -41],
    '28_-4': [-56, -44, -70],
    '28_4': [-53, -41, -72],
    '25_0': [-53, -36, -61],
    '-25_-2': [-54, -63, -37],
    '7_-2': [-42, -48, -56],
}

In [3]:
exp001_nearestneighbour_db_manual_locs  = []
exp001_nearestneighbour_db_manual_RSSIs = []
for db_item in exp001_nearestneighbour_db_manual_data_dictionary:
    x,y = map(float, db_item.split('_'))
    exp001_nearestneighbour_db_manual_locs.append([x, y])
    exp001_nearestneighbour_db_manual_RSSIs.append(exp001_nearestneighbour_db_manual_data_dictionary[db_item])

exp001_nearestneighbour_db_manual = (exp001_nearestneighbour_db_manual_RSSIs, exp001_nearestneighbour_db_manual_locs)

In [4]:
RssPosAlgo_NearestNeighbour([-42, -48, -56], exp001_nearestneighbour_db_manual)

[7.0, -2.0]

correct

In [5]:
RssPosAlgo_NearestNeighbour([-56, -44, -70], exp001_nearestneighbour_db_manual)

[28.0, -4.0]

correct

In [6]:
RssPosAlgo_NearestNeighbour([-55, -43, -69], exp001_nearestneighbour_db_manual)

[28.0, -4.0]

snaps to nearest, correct

In [7]:
RssPosAlgo_NearestNeighbour([-42, -48, -56, -2], exp001_nearestneighbour_db_manual)

ValueError: Length of the submitted RSSI array (4) for prediction does not match the length of that in the database (3)

raises error as intended, that's good

### building a database out of the recorded experiment data with kmeans

In [ ]:
datajsonpath = "../experiments/exp001_20240507_dorm_calpnp_converted_data/data-tshark/data.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["d8:84:66:39:29:e8", "d8:84:66:4a:06:c9", "d8:84:66:03:61:00"])

In [ ]:
_, _, x_train, y_train, x_test, y_test = prepare_data_loaders(inp_rss_vals, gt_locations, batch_size=1, train_test_split=0.5)

print("Shape of total RSS array:",inp_rss_vals.shape)
print("Shape of total loc array:",gt_locations.shape)
print("Shape of train and test RSS arrays, respectively:", x_train.shape, x_test.shape)
print("Shape of train and test loc arrays, respectively:", y_train.shape, y_test.shape)

In [ ]:
# get cluster IDs according to loc values, same number of clusters as manual db (=13)
num_clusters = 13
exp001_nearestneighbour_db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(x_train, y_train, num_clusters, verbose=True)

### evaluate manual vs. kmeans data dicts w.r.t. test set

In [ ]:
def geterror(xt,yt,db):
    meanerror = 0;
    for test_idx, x_test_sample in enumerate(xt): 
        loc_pred   = RssPosAlgo_NearestNeighbour(x_test_sample, db)
        meanerror += (yt[test_idx].numpy() - loc_pred)**2
    return meanerror/(test_idx+1)

In [ ]:
print("Manual L2 err:", np.linalg.norm(geterror(x_test, y_test, exp001_nearestneighbour_db_manual)))
print("Kmeans L2 err:", np.linalg.norm(geterror(x_test, y_test, exp001_nearestneighbour_db_kmeans)))

test for different split sizes

In [ ]:
splitsizes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# get cluster IDs according to loc values, same number of clusters as manual db (=13)
num_clusters = 13
for ss in splitsizes:
    _, _, x_train, y_train, x_test, y_test = prepare_data_loaders(inp_rss_vals, gt_locations, batch_size=1, train_test_split=ss)
    exp001_nearestneighbour_db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(x_train, y_train, num_clusters)
    print("Split size:", ss)
    print("Manual L2 err:", np.linalg.norm(geterror(x_test, y_test, exp001_nearestneighbour_db_manual)))
    print("Kmeans L2 err:", np.linalg.norm(geterror(x_test, y_test, exp001_nearestneighbour_db_kmeans)))
    print("")

### evaluate Kmeans normal nearest neighbour vs. interpolated

In [ ]:
_, _, x_train, y_train, x_test, y_test = prepare_data_loaders(inp_rss_vals, gt_locations, batch_size=1, train_test_split=0.5)
exp001_nearestneighbour_db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(x_train, y_train, num_clusters)
meanerror_nene_interp = 0;
meanerror_nene        = 0;
for test_idx, x_test_sample in enumerate(x_test): 
    loc_pred_nene          = RssPosAlgo_NearestNeighbour(x_test_sample, exp001_nearestneighbour_db_kmeans)
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, exp001_nearestneighbour_db_kmeans)
    meanerror_nene        += (y_test[test_idx].numpy() - loc_pred_nene)**2
    meanerror_nene_interp += (y_test[test_idx].numpy() - loc_pred_nene_interp)**2
print("NeNe       :", np.linalg.norm(meanerror_nene/(test_idx+1)))
print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))

see the improvement over ss=0.5 above